# Waitoa Fusion Audience

In [1]:
# Setup the environment

In [2]:
#pip install geopandas

In [3]:
#pip install matplotlib


#### ### Create pipeline to read all Dunnhumby data for Waitoa


In [4]:
## Do same for Waito Deli
import pandas as pd
waitoa_butchery = pd.read_csv(r'G:\My Drive\together_projects\waitoa\fusion/together-dunnhumby-butchery-data.csv')
waitoa_butchery.head(2)


C:\Users\DavidAirehrourTogeth\AppData\Local\Temp\ipykernel_44004\3566697332.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,AGG_TYPE,PRODUCT_GROUP,PRODUCT_NAME,PRODUCT_CODE,STORE_CLUSTER,FORMAT_NAME,STORE_REGION,STORE_NAME,STORE_CODE,09/09/2024_SALES,...,30/09/2024_VOLUME,07/10/2024_VOLUME,14/10/2024_VOLUME,21/10/2024_VOLUME,28/10/2024_VOLUME,04/11/2024_VOLUME,11/11/2024_VOLUME,18/11/2024_VOLUME,25/11/2024_VOLUME,02/12/2024_VOLUME
0,WEEKLY,15502 - Poultry - Butchery,NZ CHICKEN THIGHS BONE IN KG 1,5118844_KG1,NW Urban Upscale,New World,NORTH,Hobsonville New World,8232_4000,286.41,...,35.0,52.0,60.0,54.0,48.0,44.0,24.0,31.0,37.0,48.0
1,WEEKLY,Waitoa - 1550206 - Poultry Free Range - Butchery,WAITOA FR CHCKN THIGH FLT BNLS SKINLESS KG 1,5226622_KG1,<BLANK>,New World,CENTRE,Havelock North New World,8327_4000,NaN,...,NaN,1.0,1.0,2.0,1.0,6.0,NaN,NaN,13.0,18.0


In [5]:
# Get all columns that contain '_SALES' in their name
sales_column = [col for col in waitoa_butchery.columns if '_SALES' in col]

# Sum all sales columns to create total_sales
waitoa_butchery['total_sales'] = waitoa_butchery[sales_column].sum(axis=1)
# Drop all individual sales columns
waitoa_butchery = waitoa_butchery.drop(columns=sales_column)

# Get all columns that contain '_SALES' in their name
volume_column = [col for col in waitoa_butchery.columns if '_VOLUME' in col]

# Sum all sales columns to create total_sales
waitoa_butchery['sales_volume'] = waitoa_butchery[volume_column].sum(axis=1)

# Drop all individual sales columns
waitoa_butchery = waitoa_butchery.drop(columns=volume_column)

# Drop specified columns
columns_to_drop = ['AGG_TYPE', 'PRODUCT_GROUP', 'PRODUCT_CODE', 'STORE_CLUSTER','STORE_REGION']
waitoa_butchery = waitoa_butchery.drop(columns=columns_to_drop)

# Display first few rows to verify
waitoa_butchery.head()


,PRODUCT_NAME,FORMAT_NAME,STORE_NAME,STORE_CODE,total_sales,sales_volume
0,NZ CHICKEN THIGHS BONE IN KG 1,New World,Hobsonville New World,8232_4000,3808.05,636.0
1,WAITOA FR CHCKN THIGH FLT BNLS SKINLESS KG 1,New World,Havelock North New World,8327_4000,695.12,48.0
2,BIRD&BARROW CHCKN FREE RNGE WHOLE BAG KG KG 1,New World,New World St Martins,500015_11,9585.88,640.0
3,NZ CHICKEN LEGS KG 1,New World,Albany New World,6714_4000,8820.59,919.0
4,NZ CHICKEN TENDERLOINS KG 1,New World,Orewa New World,6235_4000,17575.00,2271.0


In [6]:
## In the Product_name column, separate Waitoa from the rest of competitors

# Remove records containing "INGHAMS" in PRODUCT_NAME so we can separate Waitoa from the rest of competitors
waitoa_butchery = waitoa_butchery[~waitoa_butchery['PRODUCT_NAME'].str.contains('INGHAMS', case=True, na=False)]

# Display first few rows to verify
waitoa_butchery.head()

,PRODUCT_NAME,FORMAT_NAME,STORE_NAME,STORE_CODE,total_sales,sales_volume
0,NZ CHICKEN THIGHS BONE IN KG 1,New World,Hobsonville New World,8232_4000,3808.05,636.0
1,WAITOA FR CHCKN THIGH FLT BNLS SKINLESS KG 1,New World,Havelock North New World,8327_4000,695.12,48.0
2,BIRD&BARROW CHCKN FREE RNGE WHOLE BAG KG KG 1,New World,New World St Martins,500015_11,9585.88,640.0
3,NZ CHICKEN LEGS KG 1,New World,Albany New World,6714_4000,8820.59,919.0
4,NZ CHICKEN TENDERLOINS KG 1,New World,Orewa New World,6235_4000,17575.00,2271.0


In [7]:
# Create the is_waitoa column
waitoa_butchery['is_waitoa'] = waitoa_butchery['PRODUCT_NAME'].str.contains('WAITOA', case=True, na=False)

# Create the pivot table
pivot_butchery = waitoa_butchery.pivot_table(
    index=['STORE_NAME', 'FORMAT_NAME'],
    values=['total_sales', 'sales_volume'],
    columns='is_waitoa',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Flatten the MultiIndex columns - modified to handle index columns correctly
pivot_butchery.columns = [
    col[0] if col[1] == '' else  # For index columns
    f'{col[0]}_{"waitoa" if col[1] else "competitors"}'  # For value columns
    for col in pivot_butchery.columns
]

# Reorder columns
column_order = [
    'STORE_NAME',
    'FORMAT_NAME',
    'total_sales_waitoa',
    'sales_volume_waitoa',
    'total_sales_competitors',
    'sales_volume_competitors'
]
pivot_butchery = pivot_butchery[column_order]

# Display the first few rows
pivot_butchery.head()

,STORE_NAME,FORMAT_NAME,total_sales_waitoa,sales_volume_waitoa,total_sales_competitors,sales_volume_competitors
0,Albany New World,New World,41809.10,2965.0,371353.74,37057.0
1,Albany PAKnSAVE,PAKnSAVE,15019.48,1726.0,1790108.28,188643.0
2,Alderman Drive Henderson PAKnSAVE,PAKnSAVE,4176.04,400.0,602755.95,64820.0
3,Aokautere New World,New World,3569.92,288.0,98593.03,11190.0
4,Birkenhead New World,New World,126402.32,8954.0,409495.69,44173.0


In [8]:
#pivot_butchery.to_csv('d:downloads/pivot_butchery.csv')

In [9]:
# Calculate share of sales and volume
pivot_butchery['share_of_sales'] = pivot_butchery['total_sales_waitoa'] / (pivot_butchery['total_sales_competitors'] + pivot_butchery['total_sales_waitoa'])
pivot_butchery['share_of_volume'] = pivot_butchery['sales_volume_waitoa'] / (pivot_butchery['sales_volume_competitors'] + pivot_butchery['sales_volume_waitoa'])

# Display first few rows to verify
pivot_butchery.head()


,STORE_NAME,FORMAT_NAME,total_sales_waitoa,sales_volume_waitoa,total_sales_competitors,sales_volume_competitors,share_of_sales,share_of_volume
0,Albany New World,New World,41809.10,2965.0,371353.74,37057.0,0.101193,0.074084
1,Albany PAKnSAVE,PAKnSAVE,15019.48,1726.0,1790108.28,188643.0,0.008320,0.009067
2,Alderman Drive Henderson PAKnSAVE,PAKnSAVE,4176.04,400.0,602755.95,64820.0,0.006881,0.006133
3,Aokautere New World,New World,3569.92,288.0,98593.03,11190.0,0.034943,0.025091
4,Birkenhead New World,New World,126402.32,8954.0,409495.69,44173.0,0.235870,0.168540


In [10]:
pivot_butchery_df = pivot_butchery[['STORE_NAME', 'share_of_sales', 'share_of_volume']]

In [11]:
pivot_butchery_df.head()

,STORE_NAME,share_of_sales,share_of_volume
0,Albany New World,0.101193,0.074084
1,Albany PAKnSAVE,0.008320,0.009067
2,Alderman Drive Henderson PAKnSAVE,0.006881,0.006133
3,Aokautere New World,0.034943,0.025091
4,Birkenhead New World,0.235870,0.168540


In [12]:
# # Create a merged and harmonized DataFrame by concatenating all pivot dataframes
# # Assign dataset identifiers to distinguish between them
# pivot_frozen_df = pivot_frozen_df.copy()
# pivot_frozen_df.loc[:, 'category'] = 'Frozen'

# pivot_deli_df = pivot_deli_df.copy()
# pivot_deli_df.loc[:, 'category'] = 'Deli'

# pivot_butchery_df = pivot_butchery_df.copy()
# pivot_butchery_df.loc[:, 'category'] = 'Butchery'


# # Concatenate the dataframes
# combined_df = pd.concat([pivot_frozen_df, pivot_deli_df, pivot_butchery_df], ignore_index=True)

# # Group by STORE_NAME and calculate the mean for share_of_sales and share_of_volume
# harmonized_df = combined_df.groupby('STORE_NAME')[['share_of_sales', 'share_of_volume']].mean().reset_index()

# harmonized_df.sort_values('STORE_NAME', ascending=True)


In [13]:
import geopandas as gpd

# Read the NZ stores data
nz_supermarkets  = gpd.read_file(r'G:\My Drive\together_projects\waitoa\fusion\nz-supermarket-postcodes\supermarkets-postcodes.shp')
nz_supermarkets.head(2)

,RecordID,Brand,Address,latitude,longitude,Name,RegionName,SA22025_V1,sa2_code,SA22025__2,SA32025__2,region_cod,region,population,POSTCODE,geometry
0,1,Countdown,Cnr George Bolt Memorial & Ron Keats DriveAuck...,-37.00,174.79,Auckland Airport,Auckland,147900,147900,Auckland Airport,Mangere,2,Auckland,690,2022,"POLYGON ((174.80465 -36.93684, 174.80475 -36.9..."
1,2,Countdown,76 Quay StreetAuckland,-36.85,174.77,Auckland City,Auckland,134800,134800,Auckland-University,Auckland City Centre,2,Auckland,130,1010,"MULTIPOLYGON (((174.95443 -36.73184, 174.95402..."


In [14]:
# Drop geometry field and assign to new dataframe
#nz_supermarkets_sa2 = nz_supermarkets.drop(columns=['geometry'])
#nz_supermarkets_sa2.to_csv('d:/downloads/nz-supermarkets-sa2.csv')

In [12]:
# Read the waitoa stores geo locations file
waitoa_stores_geo = pd.read_csv(r'G:\My Drive\together_projects\waitoa\fusion/waitoa_stores_geo_locations.csv')
# Display first few rows
waitoa_stores_geo.head()


206

In [16]:
import geopandas as gpd
from shapely.geometry import Point

# Convert waitoa_stores_geo to a GeoDataFrame
waitoa_stores_geo['geometry'] = waitoa_stores_geo.apply(
    lambda row: Point(row['store_lon'], row['store_lat']), axis=1
)
waitoa_stores_gdf = gpd.GeoDataFrame(waitoa_stores_geo, geometry='geometry', crs=nz_supermarkets.crs)

# Perform spatial join
waitoa_match_gdf = gpd.sjoin(waitoa_stores_gdf, nz_supermarkets, how='inner', predicate='within')

# Extract matched rows
waitoa_match_df = waitoa_match_gdf.drop(columns=['index_right']).reset_index(drop=True)

# Identify unmatched rows from nz_supermarkets
matched_indices = waitoa_match_gdf['index_right'].unique()
unmatched_nz_supermarkets = nz_supermarkets.loc[~nz_supermarkets.index.isin(matched_indices)].reset_index(drop=True)



In [17]:
# Check for duplicates based on Address
duplicates = waitoa_match_df.duplicated(subset=['RecordID'], keep='first')

# Remove duplicates and keep first occurrence
waitoa_match_df = waitoa_match_df[~duplicates].reset_index(drop=True)

# Display first few rows of deduplicated dataframe
waitoa_match_df.head()

,STORE_NAME,FORMAT_NAME,store_lat,store_lon,geometry,RecordID,Brand,Address,latitude,longitude,Name,RegionName,SA22025_V1,sa2_code,SA22025__2,SA32025__2,region_cod,region,population,POSTCODE
0,Albany New World,New World,-36.843948,174.785721,POINT (174.78572 -36.84395),2,Countdown,76 Quay StreetAuckland,-36.85,174.77,Auckland City,Auckland,134800,134800,Auckland-University,Auckland City Centre,2,Auckland,130,1010
1,Albany New World,New World,-36.843948,174.785721,POINT (174.78572 -36.84395),3,Countdown,19-25 Victoria StreetAuckland,-36.85,174.76,Auckland Victoria Street West,Auckland,133400,133400,Hobson Ridge Central,Auckland City Centre,2,Auckland,4020,1010
2,Albany New World,New World,-36.843948,174.785721,POINT (174.78572 -36.84395),63,Countdown,"AMP Centre, Ground FloorLower Albert StreetAuc...",-36.84,174.77,Albert Street,Auckland,119301,119301,Inlet Waitemata Harbour,Inlets Auckland,2,Auckland,110,1010
3,Albany New World,New World,-36.843948,174.785721,POINT (174.78572 -36.84395),230,New World,"125 Queen Street, Auckland",-36.85,174.77,New World Metro Queen Street,Upper North Island,134800,134800,Auckland-University,Auckland City Centre,2,Auckland,130,1010
4,Alderman Drive Henderson PAKnSAVE,PAKnSAVE,-36.881973,174.633139,POINT (174.63314 -36.88197),16,Countdown,West City Shopping CentreHendersonAuckland,-36.88,174.63,Henderson,Auckland,127500,127500,Henderson Central,Henderson,2,Auckland,1420,0612


### Assign calculated scores to the matched waitoa stores

In [18]:
# Create a copy of pivot_butchery_df to avoid modifying the original
harmonized_df = pivot_butchery_df.copy()

waitoa_match_scores_df = pd.merge(harmonized_df, waitoa_match_df, on='STORE_NAME', how='inner')
waitoa_match_scores_df.head()


,STORE_NAME,share_of_sales,share_of_volume,FORMAT_NAME,store_lat,store_lon,geometry,RecordID,Brand,Address,...,Name,RegionName,SA22025_V1,sa2_code,SA22025__2,SA32025__2,region_cod,region,population,POSTCODE
0,Albany New World,0.101193,0.074084,New World,-36.843948,174.785721,POINT (174.78572 -36.84395),2,Countdown,76 Quay StreetAuckland,...,Auckland City,Auckland,134800,134800,Auckland-University,Auckland City Centre,2,Auckland,130,1010
1,Albany New World,0.101193,0.074084,New World,-36.843948,174.785721,POINT (174.78572 -36.84395),3,Countdown,19-25 Victoria StreetAuckland,...,Auckland Victoria Street West,Auckland,133400,133400,Hobson Ridge Central,Auckland City Centre,2,Auckland,4020,1010
2,Albany New World,0.101193,0.074084,New World,-36.843948,174.785721,POINT (174.78572 -36.84395),63,Countdown,"AMP Centre, Ground FloorLower Albert StreetAuc...",...,Albert Street,Auckland,119301,119301,Inlet Waitemata Harbour,Inlets Auckland,2,Auckland,110,1010
3,Albany New World,0.101193,0.074084,New World,-36.843948,174.785721,POINT (174.78572 -36.84395),230,New World,"125 Queen Street, Auckland",...,New World Metro Queen Street,Upper North Island,134800,134800,Auckland-University,Auckland City Centre,2,Auckland,130,1010
4,Alderman Drive Henderson PAKnSAVE,0.006881,0.006133,PAKnSAVE,-36.881973,174.633139,POINT (174.63314 -36.88197),16,Countdown,West City Shopping CentreHendersonAuckland,...,Henderson,Auckland,127500,127500,Henderson Central,Henderson,2,Auckland,1420,0612


In [19]:
# Check for null values in each column
null_columns = waitoa_match_scores_df.isnull().sum()

# Display columns with null values (if any)
print("Columns with null values:")
print(null_columns[null_columns > 0])


Columns with null values:
RegionName    12
dtype: int64


In [20]:
# Check for duplicate rows in waitoa_match_scores_df
duplicates = waitoa_match_scores_df.duplicated()
print(f"Number of duplicate rows: {duplicates.sum()}")

# Display the duplicate rows if any exist
if duplicates.sum() > 0:
    print("\nDuplicate rows:")
    print(waitoa_match_scores_df[duplicates])
    
    # Remove duplicates and keep first occurrence
    waitoa_match_scores_df = waitoa_match_scores_df.drop_duplicates().reset_index(drop=True)
    print(f"\nShape after removing duplicates: {waitoa_match_scores_df.shape}")


Number of duplicate rows: 0


In [21]:
# Step 1: Compute average share_of_sales and share_of_volume for each POSTCODE in waitoa_match_scores_df
postcode_averages = waitoa_match_scores_df.groupby('POSTCODE')[['share_of_sales', 'share_of_volume']].mean().reset_index()

# Step 2: Merge unmatched_nz_supermarkets with the computed postcode averages
unmatched_with_postcode = unmatched_nz_supermarkets.merge(
    postcode_averages,
    on='POSTCODE',
    how='left',
    suffixes=('', '_postcode_avg')  # To distinguish from overall averages
)

# Step 3: Identify rows in unmatched_nz_supermarkets with no POSTCODE match
overall_averages = waitoa_match_scores_df[['share_of_sales', 'share_of_volume']].mean()

# Step 4: Assign fallback overall averages for rows with no POSTCODE match
unmatched_with_postcode['share_of_sales'] = unmatched_with_postcode['share_of_sales'].fillna(overall_averages['share_of_sales'])
unmatched_with_postcode['share_of_volume'] = unmatched_with_postcode['share_of_volume'].fillna(overall_averages['share_of_volume'])

# Final DataFrame
final_unmatched_nz_supermarkets = unmatched_with_postcode.copy()


In [22]:
#unmatched_with_postcode

In [23]:
# # Install required packages
# !pip install scikit-learn

In [24]:
# from sklearn.preprocessing import MinMaxScaler
# import numpy as np
# # Step 1: Concatenate waitoa_match_scores_df and final_unmatched_nz_supermarkets
# combined_stores_df = pd.concat([waitoa_match_scores_df, final_unmatched_nz_supermarkets], ignore_index=True)

# # Step 2: Aggregate population and share_of_volume at the POSTCODE level
# postcode_aggregates = combined_stores_df.groupby('POSTCODE').agg(
#     summed_population=('population', 'sum'),
#     mean_share_of_volume=('share_of_volume', 'mean')
# ).reset_index()

# # # Step 3: Normalize summed population and mean share_of_volume
# # scaler = MinMaxScaler()
# # postcode_aggregates['normalized_population'] = scaler.fit_transform(postcode_aggregates[['summed_population']])
# # postcode_aggregates['normalized_share_of_volume'] = scaler.fit_transform(postcode_aggregates[['mean_share_of_volume']])

# # # Step 4: Compute ranking score for each POSTCODE
# # postcode_aggregates['ranking_score'] = (
# #     postcode_aggregates['normalized_population'] - postcode_aggregates['normalized_share_of_volume']
# # )


# # Step 1-2 remain the same (aggregation and normalization)

# # Step 3: Compute ranking score with weighted components
# population_weight = 0.7  # 70% weight to population
# share_weight = 0.3      # 30% weight to share_of_volume

# postcode_aggregates['ranking_score'] = (
#     (population_weight * postcode_aggregates['normalized_population']) - 
#     (share_weight * postcode_aggregates['normalized_share_of_volume'])
# )

# # Step 4-7 remain the same

# # # Step 5: Rank POSTCODE groups by ranking score
# # postcode_aggregates = postcode_aggregates.sort_values(by='ranking_score', ascending=False).reset_index(drop=True)
# # postcode_aggregates['group_rank'] = postcode_aggregates.index + 1

# # # Step 6: Merge group ranks back to combined_stores_df
# # combined_stores_df = combined_stores_df.merge(
# #     postcode_aggregates[['POSTCODE', 'group_rank']],
# #     on='POSTCODE',
# #     how='left'
# # )

# # # Step 7: Sort final DataFrame by group rank and optionally within groups by ranking score
# # combined_stores_df = combined_stores_df.sort_values(by=['group_rank'], ascending=True).reset_index(drop=True)



# # Step 5: Rank POSTCODE groups by ranking score and assign to percentile brackets (1-100)
# postcode_aggregates = postcode_aggregates.sort_values(by='ranking_score', ascending=False).reset_index(drop=True)
# # Calculate percentile brackets (1-100)
# postcode_aggregates['group_rank_pct'] = np.ceil((postcode_aggregates.index + 1) / len(postcode_aggregates) * 100)

# # Step 6: Merge percentage ranks back to combined_stores_df
# combined_stores_df = combined_stores_df.merge(
#     postcode_aggregates[['POSTCODE', 'group_rank_pct']],
#     on='POSTCODE',
#     how='left'
# )

# # Step 7: Sort final DataFrame by percentage rank
# combined_stores_df = combined_stores_df.sort_values(by=['group_rank_pct'], ascending=True).reset_index(drop=True)



In [26]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
# Step 1: Concatenate waitoa_match_scores_df and final_unmatched_nz_supermarkets
combined_stores_df = pd.concat([waitoa_match_scores_df, final_unmatched_nz_supermarkets], ignore_index=True)


# Step 1: Concatenate and aggregate data
postcode_aggregates = combined_stores_df.groupby('POSTCODE').agg(
    summed_population=('population', 'sum'),
    mean_share_of_volume=('share_of_volume', 'mean')
).reset_index()

# Step 2: Normalize the population and share_of_volume columns
scaler = MinMaxScaler()
postcode_aggregates['normalized_population'] = scaler.fit_transform(postcode_aggregates[['summed_population']])
postcode_aggregates['normalized_share_of_volume'] = scaler.fit_transform(postcode_aggregates[['mean_share_of_volume']])

# Step 3: Compute ranking score with weighted components
population_weight = 0.7  # 70% weight to population
share_weight = 0.3      # 30% weight to share_of_volume

postcode_aggregates['ranking_score'] = (
    (population_weight * postcode_aggregates['normalized_population']) - 
    (share_weight * postcode_aggregates['normalized_share_of_volume'])
)

# Step 4: Rank and calculate percentiles
postcode_aggregates = postcode_aggregates.sort_values(by='ranking_score', ascending=False).reset_index(drop=True)
postcode_aggregates['group_rank_pct'] = np.ceil((postcode_aggregates.index + 1) / len(postcode_aggregates) * 100)

# Step 5: Merge back to combined_stores_df
combined_stores_df = combined_stores_df.merge(
    postcode_aggregates[['POSTCODE', 'group_rank_pct']],
    on='POSTCODE',
    how='left'
)

# Step 6: Sort final DataFrame
combined_stores_df = combined_stores_df.sort_values(by=['group_rank_pct'], ascending=True).reset_index(drop=True)

In [27]:
combined_stores_df1 = combined_stores_df.drop('geometry', axis=1)
combined_stores_df1.head()

,STORE_NAME,share_of_sales,share_of_volume,FORMAT_NAME,store_lat,store_lon,RecordID,Brand,Address,latitude,...,RegionName,SA22025_V1,sa2_code,SA22025__2,SA32025__2,region_cod,region,population,POSTCODE,group_rank_pct
0,NaN,0.134542,0.10705,NaN,NaN,NaN,262,New World,"948 Cameron Road, Tauranga",-37.71,...,Upper North Island,194601,194601,Gate Pa,Gate Pa,4,Bay of Plenty,4300,3112,1.0
1,NaN,0.134542,0.10705,NaN,NaN,NaN,69,Countdown,1368 Cameron RoadTauranga,-37.73,...,Bay of Plenty,195200,195200,Greerton North,Greerton-Poike,4,Bay of Plenty,3860,3112,1.0
2,NaN,0.134542,0.10705,NaN,NaN,NaN,382,PAK'nSAVE,"476 Cameron Road, Tauranga, 3110",-37.70,...,Upper North Island,194801,194801,Tauranga South,Tauranga South,4,Bay of Plenty,4240,3112,1.0
3,NaN,0.134542,0.10705,NaN,NaN,NaN,73,Countdown,618 Cameron Road Tauranga,-37.70,...,Bay of Plenty,194801,194801,Tauranga South,Tauranga South,4,Bay of Plenty,4240,3112,1.0
4,NaN,0.134542,0.10705,NaN,NaN,NaN,400,PAK'nSAVE,"Bromwick Street, Lower Hutt",-41.21,...,Lower North Island,244800,244800,Hutt Central South,Hutt Centre,9,Wellington,3210,5010,1.0


In [29]:
# Fill missing store_lat/store_lon values with latitude/longitude values and vice versa
combined_stores_df['store_lat'] = combined_stores_df.apply(lambda x: x['store_lat'] if pd.notnull(x['store_lat']) else x['latitude'], axis=1)
combined_stores_df['store_lon'] = combined_stores_df.apply(lambda x: x['store_lon'] if pd.notnull(x['store_lon']) else x['longitude'], axis=1)

# Fill any remaining nulls in latitude/longitude with store_lat/store_lon values
combined_stores_df['latitude'] = combined_stores_df.apply(lambda x: x['latitude'] if pd.notnull(x['latitude']) else x['store_lat'], axis=1)
combined_stores_df['longitude'] = combined_stores_df.apply(lambda x: x['longitude'] if pd.notnull(x['longitude']) else x['store_lon'], axis=1)

# Display the updated dataframe
#combined_stores_df

In [30]:
combined_stores_df.head()

,STORE_NAME,share_of_sales,share_of_volume,FORMAT_NAME,store_lat,store_lon,geometry,RecordID,Brand,Address,...,RegionName,SA22025_V1,sa2_code,SA22025__2,SA32025__2,region_cod,region,population,POSTCODE,group_rank_pct
0,NaN,0.134542,0.10705,NaN,-37.71,176.14,"POLYGON ((176.15552 -37.69744, 176.15569 -37.6...",262,New World,"948 Cameron Road, Tauranga",...,Upper North Island,194601,194601,Gate Pa,Gate Pa,4,Bay of Plenty,4300,3112,1.0
1,NaN,0.134542,0.10705,NaN,-37.73,176.13,"POLYGON ((176.15552 -37.69744, 176.15569 -37.6...",69,Countdown,1368 Cameron RoadTauranga,...,Bay of Plenty,195200,195200,Greerton North,Greerton-Poike,4,Bay of Plenty,3860,3112,1.0
2,NaN,0.134542,0.10705,NaN,-37.70,176.16,"POLYGON ((176.15552 -37.69744, 176.15569 -37.6...",382,PAK'nSAVE,"476 Cameron Road, Tauranga, 3110",...,Upper North Island,194801,194801,Tauranga South,Tauranga South,4,Bay of Plenty,4240,3112,1.0
3,NaN,0.134542,0.10705,NaN,-37.70,176.16,"POLYGON ((176.15552 -37.69744, 176.15569 -37.6...",73,Countdown,618 Cameron Road Tauranga,...,Bay of Plenty,194801,194801,Tauranga South,Tauranga South,4,Bay of Plenty,4240,3112,1.0
4,NaN,0.134542,0.10705,NaN,-41.21,174.91,"POLYGON ((174.91248 -41.14027, 174.91252 -41.1...",400,PAK'nSAVE,"Bromwick Street, Lower Hutt",...,Lower North Island,244800,244800,Hutt Central South,Hutt Centre,9,Wellington,3210,5010,1.0


### Plot the geomap of 452 stores
 

In [31]:
# !pip install geopy
# !pip install folium

In [45]:
# import libraries
from sklearn.cluster import KMeans
import numpy as np
from geopy.distance import great_circle
import folium

# Step 1: Apply KMeans clustering for exactly 100 clusters
def create_clusters_kmeans(df, n_clusters=100, min_radius_km=2):
    # Prepare coordinates for clustering
    coords = df[['latitude', 'longitude']].to_numpy()
    
    # Use KMeans for clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    df['cluster'] = kmeans.fit_predict(coords)
    
    # Calculate cluster centroids
    centroids = kmeans.cluster_centers_
    df['centroid_lat'] = df['cluster'].map(lambda c: centroids[c][0])
    df['centroid_lon'] = df['cluster'].map(lambda c: centroids[c][1])
    
    return df

# Step 2: Calculate centroids and radii for clusters
def calculate_cluster_centroids_and_radii_kmeans(df, min_radius_km):
    cluster_info = []
    
    for cluster_id, group in df.groupby('cluster'):
        # Calculate centroid
        centroid_lat = group['centroid_lat'].iloc[0]
        centroid_lon = group['centroid_lon'].iloc[0]
        centroid = (centroid_lat, centroid_lon)
        
        # Calculate maximum radius
        max_radius = max(
            great_circle(centroid, (row['latitude'], row['longitude'])).kilometers
            for _, row in group.iterrows()
        ) if len(group) > 1 else min_radius_km  # Ensure a minimum radius for single-store clusters
        
        max_radius = max(max_radius, min_radius_km)  # Apply minimum radius
        
        # Store cluster info
        cluster_info.append({
            'cluster': cluster_id,
            'centroid_lat': centroid_lat,
            'centroid_lon': centroid_lon,
            'radius_km': max_radius
        })
    
    return pd.DataFrame(cluster_info)

# Step 3: Plot clusters on a Folium map
def plot_clusters_kmeans(df, cluster_info):
    # Initialize a Folium map
    nz_map = folium.Map(location=[-40.9006, 174.8860], zoom_start=5)
    
    # Plot individual stores
    for _, row in df.iterrows():
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=f"Store: {row['Brand']}<br>Cluster: {row['cluster']}<br>Population: {row['population']}",
            icon=folium.Icon(color='blue')
        ).add_to(nz_map)
    
    # Plot cluster circles
    for _, cluster_row in cluster_info.iterrows():
        folium.Circle(
            location=[cluster_row['centroid_lat'], cluster_row['centroid_lon']],
            radius=cluster_row['radius_km'] * 1000,  # Convert to meters
            color='green',
            fill=True,
            fill_opacity=0.3,
            popup=f"Cluster: {cluster_row['cluster']}<br>Radius: {cluster_row['radius_km']:.2f} km"
        ).add_to(nz_map)
    
    # Save the map
    nz_map.save('nz_store_clusters_kmeans.html')
    return nz_map

# Step 4: Process combined_stores_df
# Create clusters and calculate cluster details
combined_stores_df = create_clusters_kmeans(combined_stores_df, n_clusters=100, min_radius_km=3)
cluster_info = calculate_cluster_centroids_and_radii_kmeans(combined_stores_df, min_radius_km=3)

# # Step 5: Plot the clusters
# nz_map = plot_clusters_kmeans(combined_stores_df, cluster_info)

# # To display the map in an interactive environment or Jupyter Notebook
# nz_map
# Step 6: Remove specific clusters
def remove_clusters(df, cluster_info, clusters_to_remove):
    # Filter out the specified clusters
    df_filtered = df[~df['cluster'].isin(clusters_to_remove)]
    cluster_info_filtered = cluster_info[~cluster_info['cluster'].isin(clusters_to_remove)]
    return df_filtered, cluster_info_filtered

# List of clusters to remove
clusters_to_remove = [49, 77, 80, 87, 88, 93, 96, 33, 34, 37, 56, 74]


# Filter out specified clusters
combined_stores_df_filtered, cluster_info_filtered = remove_clusters(
    combined_stores_df, cluster_info, clusters_to_remove
)

# Step 7: Plot the filtered clusters
nz_map_filtered = plot_clusters_kmeans(combined_stores_df_filtered, cluster_info_filtered)

# Display the updated map
nz_map_filtered


In [62]:
#nz_map_filtered.save('d:/downloads/nz_store_clusters_kmeans_filtered.html')

In [49]:
cluster_info_filtered 

,cluster,centroid_lat,centroid_lon,radius_km
0,0,-38.010000,175.330000,3.000000
1,1,-43.560000,172.730000,5.559754
2,2,-41.215455,174.880909,6.996272
3,3,-45.017500,168.722500,4.920425
4,4,-36.904444,174.702778,6.008741
...,...,...,...,...
94,94,-40.755000,175.145000,3.000000
95,95,-37.250000,174.730000,3.000000
97,97,-37.115000,175.860000,9.493130
98,98,-45.875000,170.350000,3.000000


In [51]:
combined_stores_df_filtered.head()


,STORE_NAME,share_of_sales,share_of_volume,FORMAT_NAME,store_lat,store_lon,geometry,RecordID,Brand,Address,...,SA22025__2,SA32025__2,region_cod,region,population,POSTCODE,group_rank_pct,cluster,centroid_lat,centroid_lon
0,NaN,0.134542,0.10705,NaN,-37.71,176.14,"POLYGON ((176.15552 -37.69744, 176.15569 -37.6...",262,New World,"948 Cameron Road, Tauranga",...,Gate Pa,Gate Pa,4,Bay of Plenty,4300,3112,1.0,17,-37.697397,176.132891
1,NaN,0.134542,0.10705,NaN,-37.73,176.13,"POLYGON ((176.15552 -37.69744, 176.15569 -37.6...",69,Countdown,1368 Cameron RoadTauranga,...,Greerton North,Greerton-Poike,4,Bay of Plenty,3860,3112,1.0,17,-37.697397,176.132891
2,NaN,0.134542,0.10705,NaN,-37.70,176.16,"POLYGON ((176.15552 -37.69744, 176.15569 -37.6...",382,PAK'nSAVE,"476 Cameron Road, Tauranga, 3110",...,Tauranga South,Tauranga South,4,Bay of Plenty,4240,3112,1.0,17,-37.697397,176.132891
3,NaN,0.134542,0.10705,NaN,-37.70,176.16,"POLYGON ((176.15552 -37.69744, 176.15569 -37.6...",73,Countdown,618 Cameron Road Tauranga,...,Tauranga South,Tauranga South,4,Bay of Plenty,4240,3112,1.0,17,-37.697397,176.132891
4,NaN,0.134542,0.10705,NaN,-41.21,174.91,"POLYGON ((174.91248 -41.14027, 174.91252 -41.1...",400,PAK'nSAVE,"Bromwick Street, Lower Hutt",...,Hutt Central South,Hutt Centre,9,Wellington,3210,5010,1.0,2,-41.215455,174.880909


In [54]:
## Get phyciall addresses for each cluster
from geopy.geocoders import Nominatim
from geopy.distance import great_circle
import pandas as pd

# Initialize the geolocator
geolocator = Nominatim(user_agent="geoapi_exercise")

# Step 1: Define a function to get the address for a given latitude and longitude
def get_address(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True, language='en')
        return location.address if location else None
    except Exception as e:
        print(f"Error retrieving address for ({lat}, {lon}): {e}")
        return None

# Step 2: Get addresses for all clusters in cluster_info_filtered
def get_cluster_addresses(cluster_info):
    cluster_info['address'] = cluster_info.apply(
        lambda row: get_address(row['centroid_lat'], row['centroid_lon']),
        axis=1
    )
    return cluster_info

# Step 3: Handle clusters with missing addresses
def assign_nearest_address(cluster_info, combined_stores_df):
    for index, row in cluster_info.iterrows():
        if pd.isnull(row['address']):
            # Find the nearest store to the cluster centroid
            cluster_centroid = (row['centroid_lat'], row['centroid_lon'])
            nearest_store = combined_stores_df.iloc[
                combined_stores_df.apply(
                    lambda x: great_circle(cluster_centroid, (x['latitude'], x['longitude'])).kilometers,
                    axis=1
                ).idxmin()
            ]
            # Assign the address of the nearest store
            cluster_info.at[index, 'address'] = nearest_store['Address']
    return cluster_info

# Step 4: Process cluster_info_filtered to get addresses
# First, get the physical addresses for clusters
cluster_info_with_addresses = get_cluster_addresses(cluster_info_filtered)

# Next, assign nearest store addresses to clusters with missing addresses
cluster_info_with_complete_addresses = assign_nearest_address(cluster_info_with_addresses, combined_stores_df_filtered)



C:\Users\DavidAirehrourTogeth\AppData\Local\Temp\ipykernel_55304\3547257871.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_info['address'] = cluster_info.apply(


In [55]:
cluster_info_with_complete_addresses

,cluster,centroid_lat,centroid_lon,radius_km,address
0,0,-38.010000,175.330000,3.000000,"Park Road, Te Awamutu, Waipa District, Waikato..."
1,1,-43.560000,172.730000,5.559754,"34, Glenstrae Road, Redcliffs, Linwood-Central..."
2,2,-41.215455,174.880909,6.996272,"7/2, Dowse Drive, Ava, Maungaraki, Lower Hutt,..."
3,3,-45.017500,168.722500,4.920425,"Frankton Arm Walkway, Frankton, Queenstown-Lak..."
4,4,-36.904444,174.702778,6.008741,"246A, Blockhouse Bay Road, Avondale, Whau, Auc..."
...,...,...,...,...,...
94,94,-40.755000,175.145000,3.000000,"1A, Alexander Place, Ōtaki Town, Ōtaki, Kāpiti..."
95,95,-37.250000,174.730000,3.000000,"Bowen Street, Waiuku, Franklin, Auckland, 2123..."
97,97,-37.115000,175.860000,9.493130,"Kapakapa Road, Opoutere, Thames-Coromandel Dis..."
98,98,-45.875000,170.350000,3.000000,"Taieri College, Green Street, Mosgiel, Dunedin..."


In [56]:
# Check for any null addresses in cluster_info_with_complete_addresses
null_addresses = cluster_info_with_complete_addresses['address'].isnull().sum()
print(f"Number of null addresses: {null_addresses}")


Number of null addresses: 0


In [57]:
## Add Postcodes to Cluster Information
# Step 1: Group POSTCODES by cluster
postcode_groups = (
    combined_stores_df_filtered.groupby('cluster')['POSTCODE']
    .apply(lambda x: list(set(x)))  # Remove duplicates and format as list
    .reset_index(name='postcodes')  # Reset index and rename column
)

# Step 2: Merge grouped POSTCODES into cluster_info_with_complete_addresses
cluster_info_with_postcodes = cluster_info_with_complete_addresses.merge(
    postcode_groups,
    on='cluster',
    how='left'
)

# Display the updated DataFrame with postcodes attached


In [59]:
cluster_info_with_postcodes

,cluster,centroid_lat,centroid_lon,radius_km,address,postcodes
0,0,-38.010000,175.330000,3.000000,"Park Road, Te Awamutu, Waipa District, Waikato...",[3800]
1,1,-43.560000,172.730000,5.559754,"34, Glenstrae Road, Redcliffs, Linwood-Central...","[8022, 8082, 8061, 8081]"
2,2,-41.215455,174.880909,6.996272,"7/2, Dowse Drive, Ava, Maungaraki, Lower Hutt,...","[5012, 6037, 5014, 5010]"
3,3,-45.017500,168.722500,4.920425,"Frankton Arm Walkway, Frankton, Queenstown-Lak...",[9300]
4,4,-36.904444,174.702778,6.008741,"246A, Blockhouse Bay Road, Avondale, Whau, Auc...","[0600, 0604, 1042, 1022, 0602, 1041, 1026, 102..."
...,...,...,...,...,...,...
83,94,-40.755000,175.145000,3.000000,"1A, Alexander Place, Ōtaki Town, Ōtaki, Kāpiti...",[5512]
84,95,-37.250000,174.730000,3.000000,"Bowen Street, Waiuku, Franklin, Auckland, 2123...",[2123]
85,97,-37.115000,175.860000,9.493130,"Kapakapa Road, Opoutere, Thames-Coromandel Dis...","[3620, 3579]"
86,98,-45.875000,170.350000,3.000000,"Taieri College, Green Street, Mosgiel, Dunedin...",[9024]


In [60]:
cluster_info_with_postcodes.to_csv('d:/downloads/cluster_info_with_postcodes.csv')

In [61]:
## Select columns for output for campaign 
columns_to_display = [
    'cluster',
    'sa2_code',
    'SA32025__2',
    'region',
    'POSTCODE',
    'store_lat',
    'store_lon', 
    'Brand',
    'Address',
    'share_of_sales',
    'share_of_volume',
    'population',
    'group_rank_pct',
   ]

combined_stores_df_filtered[columns_to_display].to_csv('d:/downloads/all-stores-and-rank.csv')

In [43]:
##### END #####